# Experiment 1 – Show Encrypted Data Cannot Be Read

**Goal:** Demonstrate that encrypted data stored in an AWS S3 bucket cannot be read
without the appropriate decryption key / permissions.

We will:

1. List objects in the test S3 bucket.
2. Attempt to read a Parquet file with an *encrypted* column.
3. Attempt to read the same encrypted data via Snowflake external stage.

In [ ]:
from dotenv import load_dotenv
from utility_functions import list_s3_objects, retrieve_data
from constants import BUCKET_NAME

In [10]:
# Load environment variables
load_dotenv()

True

In [11]:
# List all objects in the S3 bucket
all_keys = list_s3_objects(BUCKET_NAME)

all_parquet_files = [k for k in all_keys if k.endswith(".parquet")]
print("\nParquet objects found:")
for k in all_parquet_files:
    print(" -", k)

encrypted_candidates = [k for k in all_parquet_files if "encrypted" in k.lower()]

print("\nEncrypted candidates:")
for k in encrypted_candidates:
    print(" -", k)

Objects in bucket 'rher-s3-test-bucket':
 - employees.parquet (LastModified: 2025-11-20 14:17:33+00:00, Size: 4796 bytes)
 - sample_sensitive_data.parquet (LastModified: 2025-11-03 11:16:22+00:00, Size: 2152 bytes)
 - sample_sensitive_data_encrypted.parquet (LastModified: 2025-12-02 09:25:34+00:00, Size: 8806 bytes)

Parquet objects found:
 - employees.parquet
 - sample_sensitive_data.parquet
 - sample_sensitive_data_encrypted.parquet

Encrypted candidates:
 - sample_sensitive_data_encrypted.parquet


In [12]:
# Attempt to read the raw and encrypted data
raw_data_df = retrieve_data(BUCKET_NAME, all_parquet_files[0])
encrypted_data_df = retrieve_data(BUCKET_NAME, encrypted_candidates[0])
print(f"Raw DataFrame shape: {raw_data_df.shape}")
print(f"Encrypted DataFrame shape: {encrypted_data_df.shape}")


 successfully loaded 'employees.parquet' into DataFrame.

 successfully loaded 'sample_sensitive_data_encrypted.parquet' into DataFrame.
Raw DataFrame shape: (10, 6)
Encrypted DataFrame shape: (10, 6)


In [ ]:
# Display the raw data
raw_data_df.head()

,ID,Name,Email,Department,Salary,Password
0,1,Alice,alice@example.com,HR,55000,2o1e*OBZWW^7Kd
1,2,Bob,bob@example.com,Engineering,72000,GSxZX$zD5Qlf8^
2,3,Charlie,charlie@example.com,Marketing,63000,02QChWYlypvP#i
3,4,David,david@example.com,Finance,80000,m5AKDGR88&T*A8
4,5,Eva,eva@example.com,Engineering,75000,Yt%hUAUT2RX5$A


In [14]:
# Display the encrypted data
encrypted_data_df.head()

,ID,Name,Email,Department,Salary,Password
0,1,Alice,alice@example.com,HR,gAAAAABpLrCMmHguX0r0xqGo92_ACuSCBKuA5vtWeQzLWH...,gAAAAABpLrCMtGSy6lBrX4Ge4B_tADTvGpQkWCZ9MzGOi2...
1,2,Bob,bob@example.com,Engineering,gAAAAABpLrCMHnO3jeaL2o1_rUru3c-R75ZcIIwMbcnFO9...,gAAAAABpLrCMqaffpffNlnWpyXigdcRQs35FD0evnJoPoV...
2,3,Charlie,charlie@example.com,Marketing,gAAAAABpLrCMRUQBH41jbiG0bFtmzXordV8vZvPXuHjkhs...,gAAAAABpLrCMIeIebdmsd7i0hXysvI9OfMakQ_aSQys72G...
3,4,David,david@example.com,Finance,gAAAAABpLrCM4S53NvA6aQpgA8Z1d17dAIEpQvp_nNO1in...,gAAAAABpLrCMtj98pR40ir08dPPe10vTiJcJMu8eVpDDLy...
4,5,Eva,eva@example.com,Engineering,gAAAAABpLrCMRSI_KPTpK02SBff78xRmhqPPoE-_0r0jOR...,gAAAAABpLrCMau4X6IX1fNkwVhznV7ncx_dOuB7sQqq2Px...
